# Linear regression

In [1]:
using Flux
using Flux: Params
using DelimitedFiles: readdlm
using Statistics: mean, std

## Download data

In [2]:
isfile("housing.data") ||
    download("https://raw.githubusercontent.com/MikeInnes/notebooks/master/housing.data", "housing.data")

true

## Read data

In [3]:
rawdata = readdlm("housing.data")'

14×505 LinearAlgebra.Adjoint{Float64,Array{Float64,2}}:
   0.02731    0.02729    0.03237  …    0.06076    0.10959    0.04741
   0.0        0.0        0.0           0.0        0.0        0.0
   7.07       7.07       2.18         11.93      11.93      11.93
   0.0        0.0        0.0           0.0        0.0        0.0
   0.469      0.469      0.458         0.573      0.573      0.573
   6.421      7.185      6.998    …    6.976      6.794      6.03
  78.9       61.1       45.8          91.0       89.3       80.8
   4.9671     4.9671     6.0622        2.1675     2.3889     2.505
   2.0        2.0        3.0           1.0        1.0        1.0
 242.0      242.0      222.0         273.0      273.0      273.0
  17.8       17.8       18.7      …   21.0       21.0       21.0
 396.9      392.83     394.63        396.9      393.45     396.9
   9.14       4.03       2.94          5.64       6.48       7.88
  21.6       34.7       33.4          23.9       22.0       11.9

## The last feature (the price of the house) is our target

In [4]:
x = rawdata[1:13,:]
y = rawdata[14:14,:]

1×505 Array{Float64,2}:
 21.6  34.7  33.4  36.2  28.7  22.9  …  16.8  22.4  20.6  23.9  22.0  11.9

## Normalization

$\LARGE z = \frac{x - \mu}{\sigma}$

In [5]:
x = (x .- mean(x; dims=2)) ./ std(x; dims=2)

13×505 Array{Float64,2}:
 -0.417416  -0.417418  -0.416828  …  -0.41353   -0.407858  -0.415081
 -0.486234  -0.486234  -0.486234     -0.486234  -0.486234  -0.486234
 -0.595732  -0.595732  -1.30899       0.11315    0.11315    0.11315
 -0.272618  -0.272618  -0.272618     -0.272618  -0.272618  -0.272618
 -0.739098  -0.739098  -0.833934      0.15753    0.15753    0.15753
  0.194741   1.28121    1.01528   …   0.983996   0.725177  -0.361293
  0.366208  -0.265527  -0.808535      0.795646   0.735312   0.433641
  0.556346   0.556346   1.0759       -0.771891  -0.66685   -0.611768
 -0.868939  -0.868939  -0.754097     -0.983782  -0.983782  -0.983782
 -0.987128  -0.987128  -1.10573      -0.803294  -0.803294  -0.803294
 -0.306024  -0.306024   0.110158  …   1.17373    1.17373    1.17373
  0.441136   0.396591   0.416291      0.441136   0.403377   0.441136
 -0.494157  -1.20985   -1.36251      -0.984357  -0.866709  -0.670629

## Model

In [6]:
predict(x) = W*x .+ b

predict (generic function with 1 method)

In [7]:
W = randn(1,13)/10
b = zeros(1)

1-element Array{Float64,1}:
 0.0

## *Using CUDA*

## Loss function

$\LARGE MSE = \frac{1}{n} \sum (y - \hat{y})^2$

In [8]:
error(ŷ, y) = sum((y .- ŷ).^2)/size(y, 2)

error (generic function with 1 method)

In [9]:
loss(x, y) = error(predict(x), y)

loss (generic function with 1 method)

## Training

In [10]:
η = 0.1
θ = Params([W, b])

Params([[0.02322542840801122 -0.13784367119273286 … 0.05606042900287531 -0.2828255856841579], [0.0]])

In [11]:
function gradient_descent(θ, η, epochs)
    for i = 1:epochs
        g = gradient(() -> loss(x, y), θ)

        for t in θ
            Flux.Optimise.update!(t, g[t]*η)
        end
        @show loss(x, y)
    end
end

gradient_descent (generic function with 1 method)

In [12]:
gradient_descent(θ, η, 50)

loss(x, y) = 366.1106337485248
loss(x, y) = 241.2162899010843
loss(x, y) = 162.51118064201128
loss(x, y) = 112.35793407038291
loss(x, y) = 80.33338497248339
loss(x, y) = 59.855942952679776
loss(x, y) = 46.74217920620763
loss(x, y) = 38.3288323757638
loss(x, y) = 32.918658810260034
loss(x, y) = 29.429080437472678
loss(x, y) = 27.169031183891864
loss(x, y) = 25.697042582225638
loss(x, y) = 24.730894887692607
loss(x, y) = 24.090043200650026
loss(x, y) = 23.65889943092555
loss(x, y) = 23.36339864959412
loss(x, y) = 23.15603428332601
loss(x, y) = 23.006296193281067
loss(x, y) = 22.894557854416117
loss(x, y) = 22.80816662225014
loss(x, y) = 22.738941917892944
loss(x, y) = 22.681573580123132
loss(x, y) = 22.632595994752535
loss(x, y) = 22.589730660683557
loss(x, y) = 22.551464610800814
loss(x, y) = 22.516779876594253
loss(x, y) = 22.484979723883324
loss(x, y) = 22.455576916945702
loss(x, y) = 22.428221762586652
loss(x, y) = 22.4026556815411
loss(x, y) = 22.37868117331912
loss(x, y) = 22.35614

## Prediction

In [13]:
predict(x[:, 1]) / y[1]

1-element Array{Float64,1}:
 1.1577046102469288